Practical Session - Transformers
---

### Goal  
The goal of this session is to implement a standard Transformer in PyTorch from scratch.

### Task  
As a sample problem, we will focus on sorting a list of digits from 1 to 20.

Example:
Source Sequence: `[19, 7, 2, 9, 18]`
Target Output:   `[2, 7, 9, 18, 19]`

### Outline
1. Embed the tokens/numbers into vectors
2. The `Transformer` layer
- 2.1. Implement the self-attention layer, as seen in class
- 2.2 Integrate the self-attention layer in a transformer layer
3. Implement a `Transformer` network
4. Train the network!






---
#### 0. Generating data

In [21]:
import torch
import numpy as np

VOCAB_SIZE = 20  # Numbers from 1 to vocab_size
SEQ_LENGTH = 5   # Sequence length

def generate_data(batch_size, seq_length, vocab_size):
    """
    Generates random sequences of integers and their sorted counterparts.

    Args:
        batch_size (int): Number of sequences to generate.
        seq_length (int): Length of each sequence.
        vocab_size (int): Maximum integer value (exclusive) for the sequence elements.

    Returns:
        Tuple[torch.Tensor, torch.Tensor]:
            - src (torch.Tensor): A tensor of shape (batch_size, seq_length) containing
              random integers in the range [1, vocab_size).
            - tgt (torch.Tensor): A tensor of shape (batch_size, seq_length) containing
              the sorted version of each sequence in `src`.
    """
    src = torch.randint(1, vocab_size, (batch_size, seq_length))
    tgt = torch.sort(src, dim=1)[0]
    return src, tgt

source, target = generate_data(1, SEQ_LENGTH, VOCAB_SIZE)
print("Example source sequence and target:")
print("-" * 50)
print(f"Input Sequence:         {source.tolist()[0]}")
print(f"Expected Sorted Output: {target.tolist()[0]}")

Example source sequence and target:
--------------------------------------------------
Input Sequence:         [5, 18, 12, 14, 18]
Expected Sorted Output: [5, 12, 14, 18, 18]


---
### Step 1: Embed the tokens into vectors

First step is to transform the input integers into vectors of a fixed dimension `d`
##### How to do this:  
1. **Token Embeddings**: Each input token (integer index) is mapped to a high-dimensional vector using [`torch.nn.Embedding`](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html).  
2. **Positional Encoding**: Instead of the classical sine-cosine positional encodings, we simply use a learnable vector for each position in the sequence, again using `torch.nn.Embedding`.
3. **Summation**: The final embedding is the sum of token embeddings and positional encodings.  


In [22]:
import torch.nn as nn

class IntegerSequenceEmbedding(nn.Module):
    """
    Embedding module that combines token embeddings with positional encodings.

    Args:
        vocab_size (int): Size of the vocabulary (number of unique tokens).
        embed_dim (int): Dimension of the embeddings.
        seq_length (int): Sequence length.
    """
    def __init__(self, vocab_size=20, embed_dim=16, seq_length=5):
        super().__init__()
        # embedding layer for the tokens (numbers):
        self.token_embedding = nn.Embedding(vocab_size, embed_dim)
        # embedding layer for the positions:
        self.positional_embedding = nn.Embedding(seq_length, embed_dim)

    def forward(self, x):
        """
        Forward pass of the embedding module.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, seq_length),
                              containing integer token indices.
        Returns:
            torch.Tensor: Embedded tensor of shape (batch_size, seq_length, embed_dim).
        """
        # Token embedding
        x = self.token_embedding(x)  # Shape: (batch_size, seq_length, embed_dim)
        # Positional encoding
        positions = torch.arange(x.shape[1]).unsqueeze(0)              # Shape: (1, seq_length)
        x = x + self.positional_embedding(positions)

        return x

embedding_layer = IntegerSequenceEmbedding(vocab_size=21, embed_dim=3, seq_length=SEQ_LENGTH)
src, _ = generate_data(batch_size=1, seq_length=SEQ_LENGTH, vocab_size=VOCAB_SIZE)
embedded_src = embedding_layer(src)
print(f"Embedded batch of sequences {src.tolist()} into:")
print(f"Tensor of shape  {embedded_src.shape}")
embedded_src

Embedded batch of sequences [[16, 7, 10, 5, 15]] into:
Tensor of shape  torch.Size([1, 5, 3])


tensor([[[ 1.4250,  0.4345, -2.6929],
         [ 1.7109,  0.4940,  0.5560],
         [-1.0345, -0.0189,  1.0782],
         [ 0.3897,  0.8772, -0.4300],
         [ 0.1627, -0.4316, -0.0877]]], grad_fn=<AddBackward0>)

---
### Step 2: The `Transformer` layer

#### 2.1. Implement the self-attention layer, as seen in class

In [29]:
class SingleHeadAttention(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()

        self.W_q = nn.Linear(embed_dim, embed_dim, bias=False)  # Query projection
        self.W_k = nn.Linear(embed_dim, embed_dim, bias=False)  # Key projection
        self.W_v = nn.Linear(embed_dim, embed_dim, bias=False)  # Value projection

    def forward(self, x):
        # Input x: (batch_size, seq_length, embed_dim)
        # TODO: Complete this function
        Q = self.W_q(x)  # (batch_size, seq_length, embed_dim)
        K = self.W_k(x)  # (batch_size, seq_length, embed_dim)
        V = self.W_v(x)  # (batch_size, seq_length, embed_dim)
        d_k = K.shape[-1] # Key dimension


        # Dot-product similarities
        scores = Q @ K.transpose(1, 2)
        # Scale by dimension
        scores /= d_k ** 0.5
        # Transform the scores into probabilities with the softmax function
        scores = torch.softmax(scores, dim=-1)

        # Optional: store the attention weights for visualization
        self.attention_weights = scores

        # Update the vectors x
        x = scores @ V

        return x

# Testing
attn = SingleHeadAttention(embed_dim=128)
x = torch.randn(32, 10, 128)  # Batch of 32 sequences, each of length 10 with 128-d embeddings
output = attn(x)
print(output.shape)  # Should be (32, 10, 128)

torch.Size([32, 10, 128])


#### 2.2 Integrate the self-attention layer in a transformer layer

A **Transformer Encoder Layer** consists of:  
- A *self-attention mechanism* to capture long-range dependencies.  
- *Fully connected (feedforward) layers* to transform representations.  
- *Layer normalization* to stabilize training.  
- *Residual connections* to improve gradient flow and prevent vanishing gradients.
<div style="max-width:400px">
<img src="https://substackcdn.com/image/fetch/w_1456,c_limit,f_webp,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F5024bcc5-33c9-4d53-9bd7-56cbcf9c4627_874x1108.png" alt="Transformer Layer" />
<div/>


In [30]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        self.self_attn = SingleHeadAttention(embed_dim)
        # Normalization layers
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        # Fully connected layers
        self.fc_layers = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 2),
            nn.ReLU(),
            nn.Linear(embed_dim * 2, embed_dim)
        )


    def forward(self, x):
        # Input x: (batch_size, seq_length, embed_dim)
        # TODO: Implement encoder block, with residual connections!

        x = x + self.self_attn(self.norm1(x))
        x = x + self.fc_layers(self.norm2(x))

        return x

# Testing
attn = TransformerEncoderLayer(embed_dim=128)
x = torch.randn(32, 10, 128)  # Batch of 32 sequences, each of length 10 with 128-d embeddings
output = attn(x)
print(output.shape)  # Should be (32, 10, 128)

torch.Size([32, 10, 128])


In [31]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        self.self_attn = SingleHeadAttention(embed_dim)
        # Normalization layers
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        # Fully connected layers
        self.fc_layers = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 2),
            nn.ReLU(),
            nn.Linear(embed_dim * 2, embed_dim)
        )


    def forward(self, x):
        # Input x: (batch_size, seq_length, embed_dim)
        # TODO: Implement encoder block, with residual connections!
        y = x + self.self_attn(self.norm1(x))
        x = self.fc_layers(self.norm2(y)) + y
        #x = self.fc_layers(x)
        return x

# Testing
attn = TransformerEncoderLayer(embed_dim=128)
x = torch.randn(32, 10, 128)  # Batch of 32 sequences, each of length 10 with 128-d embeddings
output = attn(x)
print(output.shape)  # Should be (32, 10, 128)

torch.Size([32, 10, 128])


---
### 3. Implement a `Transformer` network

#### 3.1. General architecture:
The full Transformer network consists of:  
1. **Embedding Module**: Converts input tokens into dense vectors and adds positional encodings.  
2. **Transformer Layers**: A stack of self-attention layers with feedforward networks and normalization.  

<div style="max-width:600px">
<img src="https://substackcdn.com/image/fetch/w_1456,c_limit,f_webp,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2Ff6133c18-bfaf-4578-8c5a-e5ac7809f65b_1632x784.png" alt="Transformer Architecture, with zoom on transformer layer", "width="750px"\>
</div>

3. **Classification Head**: Processes the output of the Transformer layers to produce predictions.

#### 3.2. Predictions for our task

The task is to **sort a list of integers**. What should be the output of the model? Of what dimension is it?

In [32]:
import torch
import torch.nn as nn

class Transformer(nn.Module):
    def __init__(self, vocab_size, embed_dim, seq_length=5, num_layers=2):
        """
        Transformer Encoder for sequence processing.

        Args:
            vocab_size (int): Number of unique tokens in the input vocabulary.
            embed_dim (int): Dimension of the token embeddings.
            num_layers (int): Number of Transformer encoder layers.
        """
        super().__init__()

        # Token embedding layer
        self.embedding = IntegerSequenceEmbedding(vocab_size, embed_dim, seq_length)

        # Stack of Transformer Encoder Layers
        self.encoder_layers = nn.ModuleList([
            TransformerEncoderLayer(embed_dim) for i in range(num_layers)
        ])

        # Final classification head: a simple linear layer
        self.fc_out = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        """
        Forward pass of the Transformer Encoder.

        Args:
            x (Tensor): Input tensor of shape (batch_size, seq_length).

        Returns:
            Tensor: Output tensor of shape (batch_size, seq_length, vocab_size) containing probabilities for each token.
        """
        # Convert input sequence to embeddings
        x = self.embedding(x)

        # Pass through Transformer Encoder Layers
        for layer in self.encoder_layers:
            x = layer(x)

        # Apply final linear layer to get logits
        outputs = self.fc_out(x)

        return outputs

embed_dim = 32
batch_size = 16
transformer = Transformer(VOCAB_SIZE, embed_dim=embed_dim, seq_length=SEQ_LENGTH)
# Generate source and target data
source, target = generate_data(batch_size, SEQ_LENGTH, VOCAB_SIZE)

# Pass the source data through the transformer and check the output shape
outputs = transformer(source)
predictions = outputs.argmax(dim=-1) # predictions should be a list of integers, the same length as source.
print(f"Input Sequence:         {source.tolist()[0]}")
print(f"Expected Sorted Output: {target.tolist()[0]}")
print(f"Model Prediction:       {predictions.tolist()[0]}")
print("-" * 50)

Input Sequence:         [13, 15, 15, 5, 12]
Expected Sorted Output: [5, 12, 13, 15, 15]
Model Prediction:       [18, 17, 7, 18, 13]
--------------------------------------------------


In [33]:
logits.shape

torch.Size([16, 5, 5])

---
### 4. Train the network!

As for other neural networks, the Transformer parameters are learned by stochastic gradient descent on a training dataset.

In [34]:
# Data
vocab_size = 20
seq_length = 10
# Network hyperparameters
embed_dim = 32
num_layers = 2
# Training hyperparameters
batch_size = 32
num_epochs = 500

# Model, Loss, Optimizer
model = Transformer(vocab_size, embed_dim=embed_dim, seq_length=seq_length)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(num_epochs):
    # reset gradients
    optimizer.zero_grad()
    # Generate a mini-batch for training
    src, tgt = generate_data(batch_size, seq_length, vocab_size)
    # Forward pass
    output = model(src)
    loss = criterion(output.flatten(0,1), tgt.flatten()) # Why flatten the output?
    # Backward pass
    loss.backward()
    # Parameter updates
    optimizer.step()

    # Print model output at each epoch
    if epoch % 50 == 0 or epoch==10:
        test_src, test_tgt = generate_data(batch_size, seq_length, vocab_size)
        # test_pred should be a list of integers, the same length as test_src.
        test_pred = model(test_src).argmax(dim=-1)

        print(f"Epoch {epoch}")
        print(f"Input Sequence:         {test_src.tolist()[0]}")
        print(f"Expected Sorted Output: {test_tgt.tolist()[0]}")
        print(f"Model Prediction:       {test_pred.tolist()[0]}")
        print(f"Loss: {loss.item():.4f}")
        print("-" * 50)

print("Training complete!")


Epoch 0
Input Sequence:         [15, 9, 2, 15, 1, 12, 2, 17, 12, 9]
Expected Sorted Output: [1, 2, 2, 9, 9, 12, 12, 15, 15, 17]
Model Prediction:       [14, 5, 8, 10, 3, 8, 8, 15, 0, 11]
Loss: 3.3911
--------------------------------------------------
Epoch 10
Input Sequence:         [9, 17, 13, 14, 17, 17, 6, 1, 7, 9]
Expected Sorted Output: [1, 6, 7, 9, 9, 13, 14, 17, 17, 17]
Model Prediction:       [2, 3, 5, 8, 10, 10, 12, 13, 18, 19]
Loss: 2.2553
--------------------------------------------------
Epoch 50
Input Sequence:         [10, 9, 18, 14, 4, 17, 10, 19, 18, 11]
Expected Sorted Output: [4, 9, 10, 10, 11, 14, 17, 18, 18, 19]
Model Prediction:       [4, 9, 9, 10, 9, 14, 14, 17, 18, 19]
Loss: 1.1029
--------------------------------------------------
Epoch 100
Input Sequence:         [12, 12, 13, 6, 14, 13, 1, 3, 19, 5]
Expected Sorted Output: [1, 3, 5, 6, 12, 12, 13, 13, 14, 19]
Model Prediction:       [1, 3, 6, 6, 6, 13, 13, 14, 18, 19]
Loss: 0.6489
------------------------------

In [35]:
# Test the model on 5 example sequences
for i in range(5):
    print(f"Input Sequence {i + 1}:         {test_src.tolist()[i]}")
    print(f"Expected Sorted Output {i + 1}: {test_tgt.tolist()[i]}")
    print(f"Model Prediction {i + 1}:       {test_pred.tolist()[i]}")
    print("-" * 50)

print(f"Loss: {loss.item():.4f}")
print("-" * 50)

Input Sequence 1:         [4, 8, 1, 18, 12, 3, 3, 5, 6, 18]
Expected Sorted Output 1: [1, 3, 3, 4, 5, 6, 8, 12, 18, 18]
Model Prediction 1:       [1, 3, 3, 5, 6, 6, 8, 12, 18, 18]
--------------------------------------------------
Input Sequence 2:         [13, 12, 16, 16, 19, 5, 3, 6, 10, 11]
Expected Sorted Output 2: [3, 5, 6, 10, 11, 12, 13, 16, 16, 19]
Model Prediction 2:       [3, 5, 6, 10, 11, 12, 13, 16, 16, 19]
--------------------------------------------------
Input Sequence 3:         [1, 16, 8, 3, 13, 8, 1, 17, 5, 11]
Expected Sorted Output 3: [1, 1, 3, 5, 8, 8, 11, 13, 16, 17]
Model Prediction 3:       [1, 3, 3, 5, 8, 8, 11, 13, 16, 17]
--------------------------------------------------
Input Sequence 4:         [18, 9, 3, 17, 14, 15, 13, 9, 17, 10]
Expected Sorted Output 4: [3, 9, 9, 10, 13, 14, 15, 17, 17, 18]
Model Prediction 4:       [3, 9, 9, 10, 13, 14, 15, 17, 17, 18]
--------------------------------------------------
Input Sequence 5:         [7, 15, 2, 17, 9, 2, 5,

---
### 5. Implement multi-headed attention

<div style="max-width:400px">
<img src="https://substackcdn.com/image/fetch/w_1456,c_limit,f_webp,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F65c156ae-5cc5-4f7f-8652-dd5311b19beb_544x724.png" alt="Transformer Architecture, with zoom on transformer layer", "width="50px"\>
</div>


In [41]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads

        assert embed_dim % num_heads == 0, "Embedding size must be divisible by num_heads"

        self.W_q = nn.Linear(embed_dim, embed_dim)  # Query projection
        self.W_k = nn.Linear(embed_dim, embed_dim)  # Key projection
        self.W_v = nn.Linear(embed_dim, embed_dim)  # Value projection
        self.fc_out = nn.Linear(embed_dim, embed_dim)  # Output layer

    def forward(self, x):
        batch_size, seq_length, embed_dim = x.shape

        # TODO: Compute Queries, Keys, Values
        Q = self.W_q(x)  # (batch_size, seq_length, embed_dim)
        K = self.W_k(x)  # (batch_size, seq_length, embed_dim)
        V = self.W_v(x)  # (batch_size, seq_length, embed_dim)

        # Reshape for multiple heads
        Q = Q.view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        K = K.view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        V = V.view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)

        # TODO: Apply scaled dot-product attention
        # Dot-product similarities
        scores = Q @ K.transpose(-2, -1)
        # Scale by key dimension
        scores /=  K.shape[-1] ** 0.5
        # Transform the scores into probabilities with the softmax function
        scores = torch.softmax(scores, dim=-1)

        # Calculate attention output
        attn_output = scores @ V  # (batch_size, num_heads, seq_length, head_dim)

        # Reshape and apply final linear layer
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, seq_length, embed_dim)
        output = self.fc_out(attn_output)

        return output

In [42]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.self_attn = MultiHeadAttention(embed_dim, num_heads)
        # Normalization layers
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        # Fully connected layers
        self.fc_layers = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 2),
            nn.ReLU(),
            nn.Linear(embed_dim * 2, embed_dim)
        )


    def forward(self, x):
        # Input x: (batch_size, seq_length, embed_dim)
        # TODO: Implement encoder block, with residual connections!

        x = x + self.self_attn(self.norm1(x))
        x = x + self.fc_layers(self.norm2(x))

        return x

# Testing
attn = TransformerEncoderLayer(embed_dim=128, num_heads=4)
x = torch.randn(32, 10, 128)  # Batch of 32 sequences, each of length 10 with 128-d embeddings
output = attn(x)
print(output.shape)  # Should be (32, 10, 128)

torch.Size([32, 10, 128])


In [44]:
import torch
import torch.nn as nn

class Transformer(nn.Module):
    def __init__(self, vocab_size, embed_dim, seq_length=5, num_layers=2, num_heads=8): # Added num_heads here
        """
        Transformer Encoder for sequence processing.

        Args:
            vocab_size (int): Number of unique tokens in the input vocabulary.
            embed_dim (int): Dimension of the token embeddings.
            num_layers (int): Number of Transformer encoder layers.
        """
        super().__init__()

        # Token embedding layer
        self.embedding = IntegerSequenceEmbedding(vocab_size, embed_dim, seq_length)

        # Stack of Transformer Encoder Layers
        self.encoder_layers = nn.ModuleList([
            TransformerEncoderLayer(embed_dim, num_heads) for i in range(num_layers) # Pass num_heads here
        ])

        # Final classification head: a simple linear layer
        self.fc_out = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        """
        Forward pass of the Transformer Encoder.

        Args:
            x (Tensor): Input tensor of shape (batch_size, seq_length).

        Returns:
            Tensor: Output tensor of shape (batch_size, seq_length, vocab_size) containing probabilities for each token.
        """
        # Convert input sequence to embeddings
        x = self.embedding(x)

        # Pass through Transformer Encoder Layers
        for layer in self.encoder_layers:
            x = layer(x)

        # Apply final linear layer to get logits
        outputs = self.fc_out(x)

        return outputs

embed_dim = 32
batch_size = 16
transformer = Transformer(VOCAB_SIZE, embed_dim=embed_dim, seq_length=SEQ_LENGTH)
# Generate source and target data
source, target = generate_data(batch_size, SEQ_LENGTH, VOCAB_SIZE)

# Pass the source data through the transformer and check the output shape
outputs = transformer(source)
predictions = outputs.argmax(dim=-1) # predictions should be a list of integers, the same length as source.
print(f"Input Sequence:         {source.tolist()[0]}")
print(f"Expected Sorted Output: {target.tolist()[0]}")
print(f"Model Prediction:       {predictions.tolist()[0]}")
print("-" * 50)

Input Sequence:         [1, 17, 8, 2, 8]
Expected Sorted Output: [1, 2, 8, 8, 17]
Model Prediction:       [14, 14, 13, 8, 7]
--------------------------------------------------


In [45]:
# Data
vocab_size = 20
seq_length = 10
# Network hyperparameters
embed_dim = 32
num_layers = 2
# Training hyperparameters
batch_size = 32
num_epochs = 500

# Model, Loss, Optimizer
model = Transformer(vocab_size, embed_dim=embed_dim, seq_length=seq_length)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(num_epochs):
    # reset gradients
    optimizer.zero_grad()
    # Generate a mini-batch for training
    src, tgt = generate_data(batch_size, seq_length, vocab_size)
    # Forward pass
    output = model(src)
    loss = criterion(output.flatten(0,1), tgt.flatten()) # Why flatten the output?
    # Backward pass
    loss.backward()
    # Parameter updates
    optimizer.step()

    # Print model output at each epoch
    if epoch % 50 == 0 or epoch==10:
        test_src, test_tgt = generate_data(batch_size, seq_length, vocab_size)
        # test_pred should be a list of integers, the same length as test_src.
        test_pred = model(test_src).argmax(dim=-1)

        print(f"Epoch {epoch}")
        print(f"Input Sequence:         {test_src.tolist()[0]}")
        print(f"Expected Sorted Output: {test_tgt.tolist()[0]}")
        print(f"Model Prediction:       {test_pred.tolist()[0]}")
        print(f"Loss: {loss.item():.4f}")
        print("-" * 50)

print("Training complete!")


Epoch 0
Input Sequence:         [19, 14, 13, 18, 15, 18, 9, 18, 15, 19]
Expected Sorted Output: [9, 13, 14, 15, 15, 18, 18, 18, 19, 19]
Model Prediction:       [14, 4, 3, 7, 14, 13, 3, 11, 2, 5]
Loss: 3.3361
--------------------------------------------------
Epoch 10
Input Sequence:         [8, 18, 14, 1, 13, 3, 8, 19, 9, 7]
Expected Sorted Output: [1, 3, 7, 8, 8, 9, 13, 14, 18, 19]
Model Prediction:       [1, 4, 5, 8, 8, 11, 14, 14, 17, 18]
Loss: 2.1849
--------------------------------------------------
Epoch 50
Input Sequence:         [19, 5, 1, 12, 19, 17, 16, 9, 9, 1]
Expected Sorted Output: [1, 1, 5, 9, 9, 12, 16, 17, 19, 19]
Model Prediction:       [1, 1, 5, 9, 9, 9, 12, 17, 17, 19]
Loss: 0.9719
--------------------------------------------------
Epoch 100
Input Sequence:         [11, 11, 1, 14, 16, 3, 15, 7, 7, 2]
Expected Sorted Output: [1, 2, 3, 7, 7, 11, 11, 14, 15, 16]
Model Prediction:       [1, 2, 3, 7, 7, 11, 11, 14, 14, 16]
Loss: 0.5340
-----------------------------------

In [46]:
# Test the model on 5 example sequences
for i in range(5):
    print(f"Input Sequence {i + 1}:         {test_src.tolist()[i]}")
    print(f"Expected Sorted Output {i + 1}: {test_tgt.tolist()[i]}")
    print(f"Model Prediction {i + 1}:       {test_pred.tolist()[i]}")
    print("-" * 50)

print(f"Loss: {loss.item():.4f}")
print("-" * 50)

Input Sequence 1:         [6, 11, 14, 9, 18, 7, 3, 8, 5, 17]
Expected Sorted Output 1: [3, 5, 6, 7, 8, 9, 11, 14, 17, 18]
Model Prediction 1:       [3, 5, 6, 7, 8, 9, 11, 14, 17, 18]
--------------------------------------------------
Input Sequence 2:         [6, 7, 7, 17, 5, 9, 3, 4, 9, 4]
Expected Sorted Output 2: [3, 4, 4, 5, 6, 7, 7, 9, 9, 17]
Model Prediction 2:       [3, 4, 4, 4, 6, 7, 7, 9, 9, 17]
--------------------------------------------------
Input Sequence 3:         [2, 8, 18, 18, 10, 2, 12, 1, 9, 8]
Expected Sorted Output 3: [1, 2, 2, 8, 8, 9, 10, 12, 18, 18]
Model Prediction 3:       [1, 2, 2, 8, 8, 9, 10, 12, 18, 18]
--------------------------------------------------
Input Sequence 4:         [7, 2, 17, 13, 3, 12, 19, 13, 1, 12]
Expected Sorted Output 4: [1, 2, 3, 7, 12, 12, 13, 13, 17, 19]
Model Prediction 4:       [1, 2, 3, 7, 12, 12, 13, 13, 17, 19]
--------------------------------------------------
Input Sequence 5:         [14, 8, 10, 8, 1, 5, 7, 3, 7, 2]
Expected